# WSW Results

## Initialisation

Basic approach to determine the project directory

In [1]:
import os
import glob

from datetime import datetime
import time

import csv
import json

from common import Printable, projdir

from country import countryNames
from event import Event

from constants import *

## Process Years

Process all available years

In [2]:
logger = Printable()

events = {}
existingNames = {}

def initEvents():
    '''Initialise events - load entrants and index sails / GT-31s'''

    eventPaths = sorted(glob.glob(os.path.join(projdir, EVENTS_DIR, '[1-2][0-9][0-9][0-9]')))
    currYear = datetime.today().year
    for eventPath in eventPaths:
        eventYear = int(os.path.basename(eventPath))

        if eventYear == currYear:
            verbosity = appConfig['Latest']['Verbosity']
        else:
            verbosity = appConfig['History']['Verbosity']

        if eventYear == currYear or appConfig['History']['Refresh']:
            event = Event(eventPath, appConfig, existingNames=existingNames, verbosity=verbosity)
            event.initEvent()

            events[event.year] = event


def checkNationalities():
    '''Check the nationalities of entrants'''

    for year in events:
        event = events[year]

        for entrantId in event.entrants:
            entrant = event.entrants[entrantId]

            nationality = entrant.getValue('Nationality')
            if nationality != 'TBC':
                
                # The name might actually be multiple names such as "Tom Cruise + Nicole Kidman"
                names = entrant.getValue('Name').split('+')
                for name in names:
                    name = name.strip()

                    # Check nationality against existing name
                    existingName = existingNames[name]
                    if not hasattr(existingName, 'nationalities'):
                        existingName.nationalities = set()
                    if nationality not in existingName.nationalities:
                        existingName.nationalities.add(nationality)


def patchNationalities():
    '''Patch the nationalities of entrants'''

    for year in events:
        event = events[year]

        for entrantId in event.entrants:
            entrant = event.entrants[entrantId]

            nationality = entrant.getValue('Nationality')
            if nationality == 'TBC':
                
                # The name might actually be multiple names such as "Tom Cruise + Nicole Kidman"
                # Only use the first name (before the + sign) to patch the nationality
                for name in entrant.getValue('Name').split('+')[:1]:
                    name = name.strip()

                    # If there is a single value in the known list of nationalities then apply it
                    existingName = existingNames[name]
                    if hasattr(existingName, 'nationalities') and len(existingName.nationalities) == 1:
                        nationality = max(existingName.nationalities)
                        entrant.entrantDict['Nationality'] = nationality
                        entrant.entrantDict['Nation'] = countryNames[nationality]['IOC Code']


def processEvents():
    '''Process the event results'''

    for year in events:
        event = events[year]
        event.processEvent()

In [3]:
if __name__ == '__main__':
    pc1 = time.perf_counter()
    
    # Read main config
    filename = os.path.join(projdir, CONFIG_DIR, APP_CONFIG)
    with open(filename, 'r', encoding='utf-8') as f:
        jsonTxt = f.read()
        try:
            appConfig = json.loads(jsonTxt)
        except:
            logger.logError('Could not parse {}'.format(filename))
            raise
    
    print("Intialising events...")
    initEvents()
    checkNationalities()
    patchNationalities()
    print("All done!\n")

    processEvents()
    
    pc2 = time.perf_counter()
    print("Reports completed in %0.2f seconds" % (pc2 - pc1))

Intialising events...
All done!

Processing 1998...
All done!

Processing 1999...
All done!

Processing 2000...
All done!

Processing 2001...
All done!

Processing 2002...
All done!

Processing 2003...
All done!

Processing 2004...
All done!

Processing 2005...
All done!

Processing 2006...
All done!

Processing 2007...
All done!

Processing 2008...
All done!

Processing 2009...
All done!

Processing 2010...
All done!

Processing 2011...
All done!

Processing 2012...
All done!

Processing 2013...
All done!

Processing 2014...
All done!

Processing 2015...
All done!

Processing 2016...
All done!

Processing 2017...
All done!

Processing 2018...
All done!

Processing 2019...
All done!

Processing 2021...
All done!

Processing 2022...
All done!

Reports completed in 10.06 seconds


## All Done!